In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import copy
import shutil
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.11.0+cu113
Torchvision Version:  0.12.0+cu113


## Download Data

In [2]:
# Mount to GDrive to get data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change this to your own Gdrive path to where the datasets lie in
mount_dir = '/content/drive/My\ Drive/Harvard/2021-22\ Spring\ -\ CS\ 282r/CS\ 282r\ Final\ Project'

In [ ]:
# Full baseline 95 to 50 dataset
zipped_dataset = os.path.join(mount_dir, "places_validation_waterbird_95_full.zip")
!unzip $zipped_dataset -d '/content/' 

In [ ]:
# Full Gaussian 95 to 50 dataset
zipped_dataset = os.path.join(mount_dir, "places_validation_waterbird_95_gaussian_full.zip")
!unzip $zipped_dataset -d '/content/' 

In [ ]:
# Full grayscale 95 to 50 dataset
zipped_dataset = os.path.join(mount_dir, "places_validation_waterbird_95_gray_full.zip")
!unzip $zipped_dataset -d '/content/' 

In [4]:
# Full decolored 95 to 50 dataset
zipped_dataset = os.path.join(mount_dir, "places_validation_waterbird_95_decolor.zip")
!unzip $zipped_dataset -d '/content/' 

Streaming output truncated to the last 5000 lines.
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Horned_Lark_0019_73996.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Yellow_Headed_Blackbird_0049_8548.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Mourning_Warbler_0023_795362.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Horned_Lark_0014_74963.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Red_Winged_Blackbird_0072_4338.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Indigo_Bunting_0078_11852.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Bobolink_0106_9126.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/landbird/Chipping_Sparrow_0035_110138.png  
  inflating: /content/places_validation_waterbird_95_decolor/test55/lan

## Training

Code inspired by PyTorch's baseline finetuning [code](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html).

In [5]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 12

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [6]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','val','test50','test55','test60','test65','test70','test75','test80','test85','test90','test95']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Finetuning on Resnet50: baseline 50 to 95


In [ ]:
data_dir = "places_validation_waterbird_95_full"

In [ ]:
print("/train/landbird count:", len([name for name in os.listdir(os.path.join(data_dir, "train", "landbird"))]))
print("/train/waterbird count:", len([name for name in os.listdir(os.path.join(data_dir, "train", "waterbird"))]))
print("/val/landbird count:", len([name for name in os.listdir(os.path.join(data_dir, "val", "landbird"))]))
print("/val/waterbird count:", len([name for name in os.listdir(os.path.join(data_dir, "val", "waterbird"))]))
print("/test/landbird count:", len([name for name in os.listdir(os.path.join(data_dir, "test50", "landbird"))]))
print("/test/waterbird count:", len([name for name in os.listdir(os.path.join(data_dir, "test50", "waterbird"))]))

/train/landbird count: 3705
/train/waterbird count: 1090
/val/landbird count: 910
/val/waterbird count: 289
/test/landbird count: 4510
/test/waterbird count: 1284


In [ ]:
# Data augmentation and normalization for training
# Just normalization for val and test
test_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': test_transform,
    'test50': test_transform,
    'test55': test_transform,
    'test60': test_transform,
    'test65': test_transform,
    'test70': test_transform,
    'test75': test_transform,
    'test80': test_transform,
    'test85': test_transform,
    'test90': test_transform,
    'test95': test_transform,
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x]) \
                  for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(
                    image_datasets[x], 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=4) for x in [
                                            'train', 
                                            'val', 
                                            'test50',
                                            'test55',
                                            'test60',
                                            'test65',
                                            'test70',
                                            'test75',
                                            'test80',
                                            'test85',
                                            'test90',
                                            'test95'
                                            ]}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=num_epochs, 
                             is_inception=(model_name=="inception"))

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.2501 Acc: 0.9039
val Loss: 0.1196 Acc: 0.9575
test50 Loss: 0.5370 Acc: 0.7981
test55 Loss: 0.4663 Acc: 0.8202
test60 Loss: 0.4424 Acc: 0.8257
test65 Loss: 0.3876 Acc: 0.8538
test70 Loss: 0.3420 Acc: 0.8697
test75 Loss: 0.3046 Acc: 0.8852
test80 Loss: 0.2661 Acc: 0.9001
test85 Loss: 0.2015 Acc: 0.9237
test90 Loss: 0.1569 Acc: 0.9434
test95 Loss: 0.1251 Acc: 0.9558

Epoch 1/14
----------
train Loss: 0.2326 Acc: 0.9126
val Loss: 0.1205 Acc: 0.9566
test50 Loss: 0.5844 Acc: 0.7782
test55 Loss: 0.5061 Acc: 0.8058
test60 Loss: 0.4781 Acc: 0.8190
test65 Loss: 0.4205 Acc: 0.8428
test70 Loss: 0.3717 Acc: 0.8555
test75 Loss: 0.3299 Acc: 0.8787
test80 Loss: 0.2873 Acc: 0.8901
test85 Loss: 0.2137 Acc: 0.9197
test90 Loss: 0.1660 Acc: 0.9396
test95 Loss: 0.1276 Acc: 0.9520

Epoch 2/14
----------
train Loss: 0.2285 Acc: 0.9132
val Loss: 0.1204 Acc: 0.9583
test50 Loss: 0.5232 Acc: 0.8070
test55 Loss: 0.4543 Acc: 0.8328
test60 Loss: 0.4352 Acc: 0.8386
test65 Loss: 0.3759 Acc: 0.8605
test70

## Finetuning on Resnet50: baseline 50 to 95 with Gaussian augmentation


In [ ]:
data_dir = "places_validation_waterbird_95_gaussian_full"

In [ ]:
# Data augmentation and normalization for training
# Just normalization for val and test
test_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': test_transform,
    'test50': test_transform,
    'test55': test_transform,
    'test60': test_transform,
    'test65': test_transform,
    'test70': test_transform,
    'test75': test_transform,
    'test80': test_transform,
    'test85': test_transform,
    'test90': test_transform,
    'test95': test_transform,
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x]) \
                  for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(
                    image_datasets[x], 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=4) for x in [
                                            'train', 
                                            'val', 
                                            'test50',
                                            'test55',
                                            'test60',
                                            'test65',
                                            'test70',
                                            'test75',
                                            'test80',
                                            'test85',
                                            'test90',
                                            'test95'
                                            ]}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=num_epochs, 
                             is_inception=(model_name=="inception"))

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.3125 Acc: 0.8755
val Loss: 0.2080 Acc: 0.9183
test50 Loss: 0.8121 Acc: 0.6650
test55 Loss: 0.7513 Acc: 0.6950
test60 Loss: 0.6992 Acc: 0.7145
test65 Loss: 0.6219 Acc: 0.7449
test70 Loss: 0.5607 Acc: 0.7720
test75 Loss: 0.5000 Acc: 0.7984
test80 Loss: 0.4245 Acc: 0.8288
test85 Loss: 0.3610 Acc: 0.8569
test90 Loss: 0.2955 Acc: 0.8914
test95 Loss: 0.2386 Acc: 0.9096

Epoch 1/19
----------
train Loss: 0.2602 Acc: 0.8974
val Loss: 0.1575 Acc: 0.9399
test50 Loss: 0.8674 Acc: 0.6918
test55 Loss: 0.7905 Acc: 0.7180
test60 Loss: 0.7285 Acc: 0.7389
test65 Loss: 0.6411 Acc: 0.7708
test70 Loss: 0.5668 Acc: 0.7955
test75 Loss: 0.4975 Acc: 0.8195
test80 Loss: 0.4198 Acc: 0.8516
test85 Loss: 0.3395 Acc: 0.8809
test90 Loss: 0.2611 Acc: 0.9106
test95 Loss: 0.1931 Acc: 0.9348

Epoch 2/19
----------
train Loss: 0.2536 Acc: 0.9057
val Loss: 0.1401 Acc: 0.9508
test50 Loss: 0.6254 Acc: 0.7668
test55 Loss: 0.5719 Acc: 0.7879
test60 Loss: 0.5210 Acc: 0.8077
test65 Loss: 0.4710 Acc: 0.8321
test70

## Finetuning on Resnet50: baseline 50 to 95 with Grayscale augmentation


In [ ]:
data_dir = "places_validation_waterbird_95_gray_full"

In [ ]:
# Data augmentation and normalization for training
# Just normalization for val and test
test_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': test_transform,
    'test50': test_transform,
    'test55': test_transform,
    'test60': test_transform,
    'test65': test_transform,
    'test70': test_transform,
    'test75': test_transform,
    'test80': test_transform,
    'test85': test_transform,
    'test90': test_transform,
    'test95': test_transform,
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x]) \
                  for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(
                    image_datasets[x], 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=4) for x in [
                                            'train', 
                                            'val', 
                                            'test50',
                                            'test55',
                                            'test60',
                                            'test65',
                                            'test70',
                                            'test75',
                                            'test80',
                                            'test85',
                                            'test90',
                                            'test95'
                                            ]}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=num_epochs, 
                             is_inception=(model_name=="inception"))

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.2802 Acc: 0.8893
val Loss: 0.1556 Acc: 0.9466
test50 Loss: 0.5467 Acc: 0.7948
test55 Loss: 0.4899 Acc: 0.8176
test60 Loss: 0.4461 Acc: 0.8300
test65 Loss: 0.3998 Acc: 0.8469
test70 Loss: 0.3685 Acc: 0.8609
test75 Loss: 0.3055 Acc: 0.8845
test80 Loss: 0.2581 Acc: 0.9025
test85 Loss: 0.2311 Acc: 0.9089
test90 Loss: 0.1798 Acc: 0.9330
test95 Loss: 0.1440 Acc: 0.9470

Epoch 1/19
----------
train Loss: 0.2560 Acc: 0.9026
val Loss: 0.1576 Acc: 0.9483
test50 Loss: 0.5593 Acc: 0.8055
test55 Loss: 0.4994 Acc: 0.8267
test60 Loss: 0.4602 Acc: 0.8400
test65 Loss: 0.4118 Acc: 0.8547
test70 Loss: 0.3778 Acc: 0.8702
test75 Loss: 0.3162 Acc: 0.8892
test80 Loss: 0.2671 Acc: 0.9018
test85 Loss: 0.2349 Acc: 0.9122
test90 Loss: 0.1831 Acc: 0.9372
test95 Loss: 0.1480 Acc: 0.9475

Epoch 2/19
----------
train Loss: 0.2283 Acc: 0.9105
val Loss: 0.1566 Acc: 0.9483
test50 Loss: 0.5627 Acc: 0.8013
test55 Loss: 0.4980 Acc: 0.8248
test60 Loss: 0.4565 Acc: 0.8355
test65 Loss: 0.4129 Acc: 0.8540
test70

## Finetuning on Resnet50: baseline 50 to 95 with De-colored augmentation


In [9]:
data_dir = "places_validation_waterbird_95_decolor"

In [10]:
# Data augmentation and normalization for training
# Just normalization for val and test
test_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': test_transform,
    'test50': test_transform,
    'test55': test_transform,
    'test60': test_transform,
    'test65': test_transform,
    'test70': test_transform,
    'test75': test_transform,
    'test80': test_transform,
    'test85': test_transform,
    'test90': test_transform,
    'test95': test_transform,
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x]) \
                  for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(
                    image_datasets[x], 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=4) for x in [
                                            'train', 
                                            'val', 
                                            'test50',
                                            'test55',
                                            'test60',
                                            'test65',
                                            'test70',
                                            'test75',
                                            'test80',
                                            'test85',
                                            'test90',
                                            'test95'
                                            ]}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=num_epochs, 
                             is_inception=(model_name=="inception"))

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.2977 Acc: 0.8863
val Loss: 0.1309 Acc: 0.9483
test50 Loss: 0.7203 Acc: 0.7287
test55 Loss: 0.6861 Acc: 0.7454
test60 Loss: 0.5898 Acc: 0.7820
test65 Loss: 0.5234 Acc: 0.7979
test70 Loss: 0.4405 Acc: 0.8300
test75 Loss: 0.4121 Acc: 0.8528
test80 Loss: 0.3339 Acc: 0.8766
test85 Loss: 0.2714 Acc: 0.8992
test90 Loss: 0.1997 Acc: 0.9249
test95 Loss: 0.1467 Acc: 0.9460

Epoch 1/14
----------
train Loss: 0.2458 Acc: 0.9022
val Loss: 0.1517 Acc: 0.9399
test50 Loss: 0.7322 Acc: 0.7132
test55 Loss: 0.6955 Acc: 0.7313
test60 Loss: 0.6046 Acc: 0.7616
test65 Loss: 0.5399 Acc: 0.7879
test70 Loss: 0.4492 Acc: 0.8222
test75 Loss: 0.4249 Acc: 0.8405
test80 Loss: 0.3469 Acc: 0.8683
test85 Loss: 0.2768 Acc: 0.8926
test90 Loss: 0.2112 Acc: 0.9197
test95 Loss: 0.1537 Acc: 0.9470

Epoch 2/14
----------
train Loss: 0.2514 Acc: 0.9045
val Loss: 0.1847 Acc: 0.9291
test50 Loss: 0.7972 Acc: 0.7111
test55 Loss: 0.7617 Acc: 0.7325
test60 Loss: 0.6651 Acc: 0.7575
test65 Loss: 0.5955 Acc: 0.7891
test70